In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from keras.preprocessing.text import Tokenizer

from keras.utils import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D
import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/direnceran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('twitter_training.csv',sep=',', names=['Tweet_ID','Entity','Sentiment','Tweet_content']) #Verileri yükleyelim xd

data.drop_duplicates(inplace=True) 

data.dropna(axis=0, inplace=True) 
data.reset_index(inplace=True) 
data

,index,Tweet_ID,Entity,Sentiment,Tweet_content
0,0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...,...
71651,74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
71652,74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
71653,74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
71654,74680,9200,Nvidia,Positive,Just realized between the windows partition of...


In [3]:
replace_list = {r"i'm": 'i am',
                r"'re": ' are',
                r"let’s": 'let us',
                r"'s":  ' is',
                r"'ve": ' have',
                r"can't": 'can not',
                r"cannot": 'can not',
                r"shan’t": 'shall not',
                r"n't": ' not',
                r"'d": ' would',
                r"'ll": ' will',
                r"'scuse": 'excuse',
                ',': ' ,',
                '.': ' .',
                '!': ' !',
                '?': ' ?',
                '\s+': ' '}  


def clean_text(text):  
    text = text.lower()
    for s in replace_list:
        text = text.replace(s, replace_list[s])
    text = ' '.join(text.split())
    return text   

x = data['Tweet_content'].apply(lambda p: clean_text(p))

Tweet_content_len = x.apply(lambda p: len(p.split(' ')))
max_Tweet_content_len = Tweet_content_len.max()
print('max Tweet_content len: {0}'.format(max_Tweet_content_len))


max Tweet_content len: 311


In [4]:

REPLACE_WITH_SPACE = re.compile("(@)")

SPACE = " "


english_stop_words = stopwords.words('english')


In [5]:

def reviews(reviews):
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line.lower()) for line in reviews]
    return reviews



def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() if word not in english_stop_words]))
    return removed_stop_words



def get_stemmed_text(corpus):
    stemmer = PorterStemmer()

    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]



y =data['Sentiment']

In [6]:

reviewtweet = reviews(x)

no_stop_words_tweet = remove_stop_words(reviewtweet)

stemmed_reviews_tweet = get_stemmed_text(no_stop_words_tweet)

max_words = 8000

In [7]:
tokenizer = Tokenizer(  
    num_words = max_words,
    filters = '"#$%&()*+-/:;<=>@[\]^_`{|}~'
)
tokenizer.fit_on_texts(stemmed_reviews_tweet)
x = tokenizer.texts_to_sequences(stemmed_reviews_tweet)
x = pad_sequences(x, maxlen = 300)

label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(y)

y = np.array(label_tokenizer.texts_to_sequences(y))

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8) 

model_lstm = Sequential()
model_lstm.add(Embedding(input_dim = max_words, output_dim = 128, input_length = 300))
model_lstm.add(SpatialDropout1D(0.3))
model_lstm.add(LSTM(128, dropout = 0.3, recurrent_dropout = 0.3))
model_lstm.add(Dense(128, activation = 'relu'))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(5, activation = 'softmax'))
model_lstm.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

history = model_lstm.fit( 
    x_train,
    y_train,
    validation_data=(x_test,y_test) ,
    epochs = 10,
    batch_size = 512
)

model_lstm.summary()


Epoch 1/10


2023-06-13 18:28:33.547490: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


112/112 [==============================] - 319s 3s/step - loss: 1.2631 - accuracy: 0.4296 - val_loss: 1.0232 - val_accuracy: 0.5825
Epoch 2/10
112/112 [==============================] - 338s 3s/step - loss: 0.8839 - accuracy: 0.6521 - val_loss: 0.7717 - val_accuracy: 0.6961
Epoch 3/10
112/112 [==============================] - 335s 3s/step - loss: 0.6963 - accuracy: 0.7355 - val_loss: 0.6786 - val_accuracy: 0.7367
Epoch 4/10
112/112 [==============================] - 341s 3s/step - loss: 0.6040 - accuracy: 0.7720 - val_loss: 0.6527 - val_accuracy: 0.7494
Epoch 5/10
112/112 [==============================] - 341s 3s/step - loss: 0.5465 - accuracy: 0.7928 - val_loss: 0.6122 - val_accuracy: 0.7648
Epoch 6/10
112/112 [==============================] - 332s 3s/step - loss: 0.4956 - accuracy: 0.8121 - val_loss: 0.5881 - val_accuracy: 0.7797
Epoch 7/10
112/112 [==============================] - 343s 3s/step - loss: 0.4523 - accuracy: 0.8291 - val_loss: 0.5678 - val_accuracy: 0.7898
Epoch 8/10